# DataFrames II: Filtering Data

In [1]:
import pandas as pd
import datetime as dt

## This Module's Dataset + Memory Optimization
- The `pd.to_datetime` method converts a **Series** to hold datetime values.
- The `format` parameter informs pandas of the format that the times are stored in.
- We pass symbols designating the segments of the string. For example, %m means "month" and %d means day.
- The `dt` attribute reveals an object with many datetime-related attributes and methods.
- The `dt.time` attribute extracts only the time from each value in a datetime **Series**.
- Use the `astype` method to convert the values in a **Series** to another type.
- The `parse_dates` parameter of `read_csv` is an alternate way to parse strings as datetimes.

In [2]:
employees = pd.read_csv("data_files/employees.csv")

employees.head(3)
#employees.info()  # memory usage: 62.6+ KB

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,Marketing
1,Thomas,Male,3/31/1996,6:53 AM,61933,4.170,True,NaN
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance


In [3]:
employees.dtypes[["Gender", "Start Date", "Last Login Time", "Senior Management"]]

Gender               object
Start Date           object
Last Login Time      object
Senior Management    object
dtype: object

In [4]:
# format tell pandas what the format of the input is.
# i.e.
# str = "09-11-2000"   format = "%m-%d-%Y"
# str = "09/11/2000"   format = "%m/%d/%Y"

employees["Start Date"] = pd.to_datetime( employees["Start Date"], format="%m/%d/%Y" )

In [5]:
# %H = Hour
# %M = Minute
# %p = am/pm

# NOTE: In the absence of a date, pandas will default to 1900-01-01
# because it just store both a date and a time.
# To get around this, we use .dt.time after the to_datetime function
employees["Last Login Time"] = pd.to_datetime( employees["Last Login Time"], format="%H:%M %p" ).dt.time

In [6]:
employees["Senior Management"] = employees["Senior Management"].astype(bool)

In [7]:
employees["Gender"] = employees["Gender"].astype("category")

In [8]:
employees.head(3)

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance


In [9]:
employees.dtypes[["Gender", "Start Date", "Last Login Time", "Senior Management"]]

#employees.info()  # memory usage: 49.1+ KB (21.5% optimization)

Gender                     category
Start Date           datetime64[ns]
Last Login Time              object
Senior Management              bool
dtype: object

---

## Filter a DataFrame Based On a Single Condition
- Pandas needs a **Series** of Booleans to perform a filter.
- Pass the Boolean Series inside square brackets after the **DataFrame**.
- We can generate a Boolean Series using a wide variety of operations (equality, inequality, less than, greater than, inclusion, etc)

In [10]:
employees[employees["Gender"] == "Male"]  # Return all male employees

employees[employees["Team"] == "Finance"]  # Return all employees that are in finance

# You can also break up the syntax for readability
on_marketing_team = employees["Team"] == "Marketing"
employees[on_marketing_team]

# For columns that are already a bool - such as "Senior Management" - we can skip the operator
employees[employees["Senior Management"]]

employees[employees["Salary"] > 100_000]

employees[employees["Bonus %"] < 1.5].head()


# Pandas is smart and (if using the same format) will understand date times
employees[ employees["Start Date"] < "1995-01-01" ]

# Hour, Minute, Second
time = dt.time(8, 0, 0)
employees[ employees["Last Login Time"] > time ]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
7,NaN,Female,2015-07-20,10:43:00,45906,11.598,True,Finance
10,Louise,Female,1980-08-12,09:01:00,63241,15.132,True,NaN
13,Gary,Male,2008-01-27,11:40:00,109831,5.831,False,Sales
...,...,...,...,...,...,...,...,...
983,John,Male,1982-12-23,10:35:00,146907,11.738,False,Engineering
985,Stephen,NaN,1983-07-10,08:10:00,85668,1.909,False,Legal
988,Alice,Female,2004-10-05,09:34:00,47638,11.209,False,Human Resources
992,Anthony,Male,2011-10-16,08:35:00,112769,11.625,True,Finance


---

## Filter with More than One Condition (AND)
- Add the `&` operator in between two Boolean **Series** to filter by multiple conditions.
- We can assign the **Series** to variables to make the syntax more readable.

In [11]:
# NOTE: For whatever reason, you must put the operators into a variable before using them.

is_female = employees["Gender"] == "Female"
is_marketing = employees["Team"] == "Marketing"

employees[ is_female & is_marketing ].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
43,Marilyn,Female,1980-12-07,03:16:00,73524,5.207,True,Marketing
62,NaN,Female,2007-06-12,05:25:00,58112,19.414,True,Marketing
98,Tina,Female,2016-06-16,07:47:00,100705,16.961,True,Marketing
140,Shirley,Female,1981-02-28,01:23:00,113850,1.854,False,Marketing
158,Norma,Female,1999-02-28,08:45:00,114412,8.756,True,Marketing


In [12]:
is_male = employees["Gender"] == "Male"
is_engineer = employees["Team"] == "Engineering"
over_paid = employees["Salary"] > 130_000

employees[ is_male & is_engineer & over_paid ].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
171,Patrick,Male,2007-08-17,03:16:00,143499,17.495,True,Engineering
175,Willie,Male,1998-02-17,08:20:00,146651,1.451,True,Engineering
447,Gregory,Male,2009-05-15,03:52:00,142208,11.204,True,Engineering
604,Bruce,Male,2013-03-15,11:13:00,141335,15.427,True,Engineering
652,Willie,Male,2009-12-05,05:39:00,141932,1.017,True,Engineering


---

## Filter with More than One Condition (OR)
- Use the `|` operator in between two Boolean **Series** to filter by *either* condition.

In [13]:
# Find employees that are Senior Management or started before 1990

is_senior = employees["Senior Management"]
started_before_1990 = employees["Start Date"] < "1990-01-01"

employees[ is_senior | started_before_1990 ].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal


## Filter using both & and |

In [14]:
# First name Robbert AND works in Client Services, OR Start Date after 2016-06-01

name = employees["First Name"] == "Robert"
role = employees["Team"] == "Client Services"
start = employees["Start Date"] > "2016-06-01"


# NOTE: You MUST have () to specify the first item you want
# to evaluate, then the second operation will happen.

employees[ (name & role) | start ]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
15,Lillian,Female,2016-06-05,06:09:00,59414,1.256,False,Product
98,Tina,Female,2016-06-16,07:47:00,100705,16.961,True,Marketing
387,Robert,Male,1994-10-29,04:26:00,123294,19.894,False,Client Services
451,Terry,NaN,2016-07-15,12:29:00,140002,19.490,True,Marketing


---

## The isin Method
- The `isin` **Series** method accepts a collection object like a list, tuple, or **Series**.
- The method returns True for a row if its value is found in the collection.

In [15]:
legal_team = employees["Team"] == "Legal"
sales_team = employees["Team"] == "Sales"
product_team = employees["Team"] == "Product"

# Find all the people that are on either the legal, sales, or product team.
employees.loc[legal_team | sales_team | product_team].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
6,Ruby,Female,1987-08-17,04:20:00,65476,10.012,True,Product
11,Julie,Female,1997-10-26,03:19:00,102508,12.637,True,Legal
13,Gary,Male,2008-01-27,11:40:00,109831,5.831,False,Sales
15,Lillian,Female,2016-06-05,06:09:00,59414,1.256,False,Product


In [16]:
# The `isin` method makes the code above much more simple.
# NOTE: This returns a series.

employees["Team"].isin(["Legal", "Sales", "Product"])
"""
--- Output ---
995    False
996    False
997     True
998    False
999     True
"""

# This will output a dataframe with all the employees that fit the given description.
employees.loc[ employees["Team"].isin(["Legal", "Sales", "Product"]) ].head()

# NOTE: This is also more efficient since pandas doesnt have to iterate over
#       the whole database 3 separate times.
#       Also note, the isin method ONLY works with a single column. ie employees["Team"]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
6,Ruby,Female,1987-08-17,04:20:00,65476,10.012,True,Product
11,Julie,Female,1997-10-26,03:19:00,102508,12.637,True,Legal
13,Gary,Male,2008-01-27,11:40:00,109831,5.831,False,Sales
15,Lillian,Female,2016-06-05,06:09:00,59414,1.256,False,Product


---

## The isnull and notnull Methods
- The `isnull` method returns True for `NaN` values in a **Series**.
- The `notnull` method returns True for present values in a **Series**.

In [17]:
# Will return True for NaN values
employees["Team"].isnull().head()

0    False
1     True
2    False
3    False
4    False
Name: Team, dtype: bool

In [18]:
# Will return True for NON-NaN values
employees["Team"].notnull().head()

0     True
1    False
2     True
3     True
4     True
Name: Team, dtype: bool

In [19]:
employees.loc[ (employees["First Name"].isnull()) & (employees["Team"].notnull()) ].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
7,NaN,Female,2015-07-20,10:43:00,45906,11.598,True,Finance
25,NaN,Male,2012-10-08,01:12:00,37076,18.576,True,Client Services
39,NaN,Male,2016-01-29,02:33:00,122173,7.797,True,Client Services
51,NaN,NaN,2011-12-17,08:29:00,41126,14.009,True,Sales
62,NaN,Female,2007-06-12,05:25:00,58112,19.414,True,Marketing


---

## The between Method
- The `between` method returns True if a **Series** value is found within its range.

In [20]:
# Soyboy way 👎
employees.loc[ (employees["Salary"] >= 60_000) & (employees["Salary"] <= 80_000) ]


# Giga chad way 👍
employees.loc[ employees["Salary"].between(60_000, 70_000) ].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
6,Ruby,Female,1987-08-17,04:20:00,65476,10.012,True,Product
10,Louise,Female,1980-08-12,09:01:00,63241,15.132,True,NaN
20,Lois,NaN,1995-04-22,07:18:00,64714,4.934,True,Legal
41,Christine,NaN,2015-06-28,01:08:00,66582,11.308,True,Business Development


In [21]:
employees.loc[ employees["Bonus %"].between(10.0, 11.0) ].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
6,Ruby,Female,1987-08-17,04:20:00,65476,10.012,True,Product
97,Laura,NaN,2014-07-19,09:23:00,140371,10.620,True,Marketing
132,Carlos,Male,1995-01-04,07:02:00,146670,10.763,False,Human Resources
139,NaN,Female,1990-10-03,01:08:00,132373,10.527,True,NaN


In [22]:
# NOTE: The between method is inclusive meaning if you used 1991-01-01
#       it would also pick up poeple on that date

employees.loc[ employees["Start Date"].between("1990-01-01", "1990-12-31") ].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
52,Todd,Male,1990-02-18,02:41:00,49339,1.695,True,Human Resources
64,Kathleen,NaN,1990-04-11,06:46:00,77834,18.771,False,Business Development
139,NaN,Female,1990-10-03,01:08:00,132373,10.527,True,NaN
141,Adam,Male,1990-12-24,08:57:00,110194,14.727,True,Product
163,Terry,Male,1990-09-03,09:15:00,52226,19.135,False,Client Services


In [23]:
# NOTE: For time it must be a datetime.time data type
import datetime as dt

eight_thirty = dt.time(8, 30, 0)
noon = dt.time(12, 0, 0)

employees.loc[ employees["Last Login Time"].between(eight_thirty, noon) ].head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
7,NaN,Female,2015-07-20,10:43:00,45906,11.598,True,Finance
10,Louise,Female,1980-08-12,09:01:00,63241,15.132,True,NaN
13,Gary,Male,2008-01-27,11:40:00,109831,5.831,False,Sales
18,Diana,Female,1981-10-23,10:27:00,132940,19.082,False,Client Services


---

## The duplicated Method
- The `duplicated` method returns True if a **Series** value is a duplicate.
- Pandas will mark one occurrence of a repeated value as a non-duplicate.
- Use the `keep` parameter to designate whether the first or last occurrence of a repeated value should be considered the "non-duplicate".
- Pass False to the `keep` parameter to mark all occurrences of repeated values as duplicates.
- Use the tilde symbol (`~`) to invert a **Series's** values. Trues will become Falses, and Falses will become trues.

In [24]:
"""
The way duplicated works is the first time a name shows up
ie `Douglas` it will be marked as False, because that's the first
time that name has showed up. Now, any time Douglas comes up
again, that name will be marked as True.
"""

emp_copy = employees.copy()

emp_copy["First Name"].duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
995     True
996     True
997     True
998     True
999     True
Name: First Name, Length: 1000, dtype: bool

In [25]:
# Note This will only give you dup values because non-dups return false.
emp_copy.loc[ emp_copy["First Name"].duplicated() ]


# This will do the opposite, it will keep the first accurace and remove the last.
# This will return True for the first accurace, and False for the dups.
# non-dups now return True.
emp_copy.loc[ emp_copy["First Name"].duplicated(keep="last") ]


# This will mark ALL dupes as True. ie now ALL accuraces of Douglas will be true
# from the first to last.
# All completely unique names will be false.
emp_copy.loc[ emp_copy["First Name"].duplicated(keep=False) ]


# The `~` key will invert any value. ie True becomes False, and vice versa.
# This will do the opposite of keep=False
# This will mark all completely unique names as True
emp_copy.loc[ ~emp_copy["First Name"].duplicated(keep=False) ]


,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
8,Angela,Female,2005-11-22,06:29:00,95570,18.523,True,Engineering
33,Jean,Female,1993-12-18,09:07:00,119082,16.180,False,Business Development
190,Carol,Female,1996-03-19,03:39:00,57783,9.129,False,Finance
291,Tammy,Female,1984-11-11,10:30:00,132839,17.463,True,Client Services
495,Eugene,Male,1984-05-24,10:54:00,81077,2.117,False,Sales
688,Brian,Male,2007-04-07,10:47:00,93901,17.821,True,Legal
832,Keith,Male,2003-02-12,03:02:00,120672,19.467,False,Legal
887,David,Male,2009-12-05,08:48:00,92242,15.407,False,Legal


---

## The drop_duplicates Method
- The `drop_duplicates` method deletes rows with duplicate values.
- By default, it will remove a row if *all* of its values are shared with another row.
- The `subset` parameter configures the columns to look for duplicate values within.
- Pass a list to `subset` parameter to look for duplicates across multiple columns.

In [26]:
# By default drop_duplicates will only drop it if ALL the values
# on a row are the exact same as another.
emp_copy.drop_duplicates()


# To search through a column, you would use `subset`
emp_copy.drop_duplicates(subset="Team")

# Keep the last accurace
emp_copy.drop_duplicates(subset="Team", keep="last")

# Drop all rows if it has a dup
emp_copy.drop_duplicates(subset="First Name", keep=False)



# This will look for the *combination* of the values.
# ie Team=Business Development  AND  Senior Management=True will only show up once,
# but Team=Business Development  AND  Senior Management=False can show up,
# because the value for Team AND Senior Managment are different.
emp_copy.drop_duplicates(subset=["Senior Management", "Team"])

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
6,Ruby,Female,1987-08-17,04:20:00,65476,10.012,True,Product
8,Angela,Female,2005-11-22,06:29:00,95570,18.523,True,Engineering
9,Frances,Female,2002-08-08,06:51:00,139852,7.524,True,Business Development
11,Julie,Female,1997-10-26,03:19:00,102508,12.637,True,Legal


---

## The unique and nunique Methods
- The `unique` method on a **Series** returns a collection of its unique values. The method does not exist on a **DataFrame**.
- The `nunique` method returns a *count* of the number of unique values in the **Series**/**DataFrame**.
- The `dropna` parameter configures whether to include or exclude missing (`NaN`) values.

In [27]:
# List all the unique values
employees["Gender"].unique()

# Return a count of valid unique values. WITHOUT NaN values
employees["Gender"].nunique()

2

In [28]:
employees["Team"].unique()

employees["Team"].nunique()

10